In [1]:
import numpy as np
import LoadData as ld
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import mne
from time import perf_counter
import keras
import tensorflow as tf


def createEpoch(feature_data, epoch_size, epoch_window):
    '''
    this creates epoch for CSP data which will be lateer
    used for LDA
    '''
    epochs = []
    n_samples = feature_data.shape[1]
    for i in range(0, n_samples, epoch_window):
        if n_samples - i < epoch_size:
            break

        epoch_data = feature_data[:, i: i + epoch_size]
        epochs.append(epoch_data)
    
    
    epochs_array = np.array(epochs)

    return epochs_array



def createChannelInfo(no_channels):
    '''This function creates an info object 
    that is required later to convert the eeg data from np data
    to mne.io.raw data.

    no_channels: number of channels used in the eeg data

    returns info object
    '''
    ch_names = []
    for i in range(no_channels):
        ch_names.append('Channel' +"-"+ str(i))   #for this code I have just created dummy channel names. We can have actual channel names
    info = mne.create_info(ch_names = ch_names, sfreq = 250, ch_types='eeg') 
    return info 

def bandPassFilterReshape(raw_data, info):
    '''
    First converts the np data into mne.io.raw data and
    uses createEpoch funcn to convert the data into epochs
    raw_data : eeg data
    info: info object we created before which has information about the channel
    
    returns epoch data which will be applied for CSP
    '''
    raw_data = np.transpose(raw_data)
    features = mne.io.RawArray(raw_data, info)
    features.filter(7.0, 40.0)
    features_data = features.get_data()
    # features_data = np.transpose(features_data)
    features_data = createEpoch(features_data, 250, 64)
    return features_data


In [21]:
raw_data_hand = ld.loadData()   
raw_data_feet = ld.loadData()    #loading eeg data of any two movements of ur choosing

test_data_hand = ld.loadData()
test_data_feet = ld.loadData() 

info = createChannelInfo(22)     #we will need the info object to convert the eeg data from np data to raw_eeg data which we will need for bandpass filtering

features_hand = bandPassFilterReshape(raw_data_hand, info)   #this converts the eeg data into epochs, which will be needed for CSP
features_feet = bandPassFilterReshape(raw_data_feet, info)

features_test_hand = bandPassFilterReshape(test_data_hand, info)
features_test_feet = bandPassFilterReshape(test_data_feet, info)

The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
Creating RawArray with float64 data, n_channels=22, n_times=51000
    Range : 0 ... 50999 =      0.000 ...   203.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)

Creating RawArray with float64 data, n_channels=22, n_times=50250
    Range : 0 ... 50249 =      0.000 ...   200.996 secs
Read

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Creating RawArray with float64 data, n_channels=22, n_times=53250
    Range : 0 ... 53249 =      0.000 ...   212.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [22]:
features =  np.concatenate((features_hand, features_feet))
label = np.concatenate((np.zeros(features_hand.shape[0]),np.ones(features_feet.shape[0])))

features_test =  np.concatenate((features_test_hand, features_test_feet))
label_test = np.concatenate((np.zeros(features_test_hand.shape[0]),np.ones(features_test_feet.shape[0])))

In [23]:
csp = mne.decoding.CSP(n_components = 21, reg = None, log = True, norm_trace = False)
features_transform = csp.fit_transform(features, label)
features_test_transform = csp.transform(features_test)
lda = LinearDiscriminantAnalysis()    #creating LDA classifier (we can specify the dimension, but if only two movements are classified, it is going to be automatically 1 dimension)
lda_transform = lda.fit_transform(features_transform, label)
lda_transform_test = lda.transform(features_test_transform)
lda_transform = np.reshape(lda_transform, (lda_transform.shape[0]))
lda_transform_test = np.reshape(lda_transform_test, (lda_transform_test.shape[0]))

dataset = tf.data.Dataset.from_tensor_slices((lda_transform, label))
dataset = dataset.shuffle(buffer_size= lda_transform.shape[0]).batch(32)

dataset_test = tf.data.Dataset.from_tensor_slices((lda_transform_test, label_test))
dataset_test = dataset_test.shuffle(buffer_size = lda_transform_test.shape[0]).batch(32)




Computing rank from data with rank=None


    Using tolerance 55 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 55 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.


In [24]:
input = keras.Input(shape=(1,))

hl1 =  keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(input)

hl2 = keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(hl1)

hl3 = keras.layers.Dense(128, activation = keras.layers.LeakyReLU(0.3)) (hl2)


output = keras.layers.Dense(2, activation = "softmax")(hl3)


model =  keras.Model(input, output)
 
print(model.summary())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,538 (131.01 KB)

 Trainable params: 33,538 (131.01 KB)

 Non-trainable params: 0 (0.00 B)

None


In [25]:
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

model.fit(dataset, epochs = 50, validation_data = dataset_test)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8231 - loss: 0.4132 - val_accuracy: 0.6726 - val_loss: 0.7359
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8476 - loss: 0.3326 - val_accuracy: 0.6744 - val_loss: 0.7447
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8695 - loss: 0.2979 - val_accuracy: 0.6738 - val_loss: 0.7204
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8643 - loss: 0.3109 - val_accuracy: 0.6714 - val_loss: 0.7176
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8556 - loss: 0.3299 - val_accuracy: 0.6726 - val_loss: 0.7339
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8597 - loss: 0.3138 - val_accuracy: 0.6714 - val_loss: 0.7173
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8776 - loss: 0.3040 - val_accuracy: 0.6683 - val_loss: 0.7062
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8478 - loss: 0.3292 - val_accuracy: 0.6720 - val_loss: